In [1]:
import os
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset
import torchvision.datasets as datasets
import numpy as np
import torch.nn as nn
import torch.optim as optim
import cv2
import glob
from torchvision import models
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from torch.utils.tensorboard import SummaryWriter
from torch.utils.tensorboard import summary as summary_lib
from PIL import Image  # Add this import
import torchvision.models as models

In [2]:
# Check if CUDA is available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cuda


In [3]:
transform = transforms.Compose([transforms.Resize((227, 227)), transforms.ToTensor()])

In [4]:
train_dataset = datasets.CIFAR10(root='./data', train=False, transform=transform, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=1, shuffle=True)

Files already downloaded and verified


In [5]:
min_sigma = 0
max_sigma = 50/255

In [6]:
def add_noise(image, sigma):
    noise = torch.randn_like(image) * sigma
    noisy_image = torch.clamp(image + noise, 0, 1)
    return noisy_image

In [7]:
def save_noisy_image(img, name):
    img = img.view(img.size(0), 3, 227, 227)
    torchvision.utils.save_image(img, name)

In [8]:

from PIL import Image

class CustomDataset(Dataset):
    def __init__(self, image_folder, sigma_values, transform=None, device=torch.device("cpu")):
        self.image_paths = glob.glob(os.path.join(image_folder, "*.png"))  # Adjust this for your image format
        self.sigma_values = sigma_values
        self.transform = transform
        self.device = device

    def __len__(self):
        return len(self.sigma_values)

    def __getitem__(self, index):
        image_path = self.image_paths[index]
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = Image.fromarray(image)  # Convert NumPy array to PIL Image
        if self.transform:
            image = self.transform(image)
        sigma = torch.tensor(self.sigma_values[index], dtype=torch.float32, device=self.device)  # Ensure sigma is float32
        return image.to(self.device), sigma

In [9]:
def customFunction():
    sigma_values = []
    i = 1
    j = 1
    for inputs, _ in train_loader:
        sigma = np.random.uniform(min_sigma, max_sigma)
        sigma_values.append(sigma)
        noisy_inputs = add_noise(inputs, sigma)
        noisy_image = (noisy_inputs.squeeze().numpy() * 255).astype(np.uint8)
        if i <= 100:
            new_image = torch.cat((inputs, noisy_inputs), 0)
            save_noisy_image(new_image, f"C:/Users/IICT3/Bithi/Noise regression/Output100/{j}.png")
        i += 1
        save_noisy_image(noisy_inputs, f"C:/Users/IICT3/Bithi/Noise regression/Noisy image/{j}.png")
        j += 1
    sigma_values = np.array(sigma_values)
    return sigma_values

In [10]:
sigma_values = customFunction()

In [11]:
input_dir = "C:/Users/IICT3/Bithi/Noise regression/Noisy image"
dataset = CustomDataset(image_folder=input_dir, sigma_values=sigma_values, transform=transform, device=device)

In [12]:
# Cross-Validation setup
kf = KFold(n_splits=5, shuffle=True, random_state=42)

In [13]:
def train(model, train_loader, criterion, optimizer, epoch, writer):
    model.train()
    running_loss = 0.0
    predictions = []
    targets = []

    for batch_idx, (inputs, targets_batch) in enumerate(train_loader):
        inputs = inputs.to(device).float()  # Convert inputs to float32
        targets_batch = targets_batch.unsqueeze(1).to(device).float()  # Convert targets to float32
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predictions.extend(outputs.cpu().detach().numpy())
        targets.extend(targets_batch.cpu().detach().numpy())

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch} [{batch_idx * len(inputs)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')

    avg_loss = running_loss / len(train_loader)
    print(f'Train Epoch: {epoch}\tAverage Loss: {avg_loss:.6f}')
    mae, rmse, r2 = calculate_metrics(predictions, targets)
    print(f'MAE: {mae:.6f}, RMSE: {rmse:.6f}, R2 Score: {r2:.6f}')
    writer.add_scalar('Training Loss', avg_loss, epoch)
    writer.add_scalar('Training MAE', mae, epoch)
    writer.add_scalar('Training RMSE', rmse, epoch)
    writer.add_scalar('Training R2 Score', r2, epoch)


In [14]:
def test(model, test_loader, criterion, epoch, writer):
    model.eval()
    test_loss = 0.0
    predictions = []
    targets = []

    with torch.no_grad():
        for inputs, targets_batch in test_loader:
            inputs = inputs.to(device).float()  # Convert inputs to float32
            targets_batch = targets_batch.unsqueeze(1).to(device).float()  # Convert targets to float32
            outputs = model(inputs)
            test_loss += criterion(outputs, targets_batch).item()

            predictions.extend(outputs.cpu().detach().numpy())
            targets.extend(targets_batch.cpu().detach().numpy())

    avg_loss = test_loss / len(test_loader)
    print(f'Test Epoch: {epoch}\tAverage Loss: {avg_loss:.6f}')
    mae, rmse, r2 = calculate_metrics(predictions, targets)
    print(f'MAE: {mae:.6f}, RMSE: {rmse:.6f}, R2 Score: {r2:.6f}')
    writer.add_scalar('Testing Loss', avg_loss, epoch)
    writer.add_scalar('Testing MAE', mae, epoch)
    writer.add_scalar('Testing RMSE', rmse, epoch)
    writer.add_scalar('Testing R2 Score', r2, epoch)
    #plot_predictions(predictions, targets, epoch)

In [15]:
def calculate_metrics(predictions, targets):
    predictions = np.array(predictions).reshape(-1)
    targets = np.array(targets).reshape(-1)
    mae = mean_absolute_error(targets, predictions)
    rmse = mean_squared_error(targets, predictions, squared=False)
    r2 = r2_score(targets, predictions)
    return mae, rmse, r2

In [16]:
def plot_predictions(predictions, targets, epoch):
    plt.figure(figsize=(10, 5))
    plt.scatter(targets, predictions, alpha=0.5)
    plt.xlabel('Ground Truth')
    plt.ylabel('Predictions')
    plt.title(f'Predictions vs. Ground Truth (Epoch {epoch})')
    plt.grid(True)
    plt.savefig(f'predictions_vs_ground_truth_epoch_{epoch}.png')
    plt.show()

In [17]:
import glob


In [18]:
import warnings

# Ignore specific warnings
warnings.filterwarnings("ignore", message="The parameter 'pretrained' is deprecated")
warnings.filterwarnings("ignore", message="Arguments other than a weight enum or `None` for 'weights' are deprecated")



In [ ]:
#VGG (original)
class NoiseNet(nn.Module):
    def __init__(self):
        super(NoiseNet, self).__init__()
        self.model = models.vgg16(pretrained=True)
        
        self.model.classifier = nn.Sequential(
            nn.Dropout(p=0.5),
            nn.Linear(in_features=25088, out_features=4096, bias=True),
            nn.ReLU(),
            nn.Linear(in_features=4096, out_features=1000, bias=True),
            nn.ReLU(),
            nn.Linear(in_features=1000, out_features=1, bias=True)
        )

    def forward(self, x):
        return self.model(x)
    
#summary(model2,(3,227,227))
model=NoiseNet()
model

In [19]:
#VGG (Fine tuned)

class NoiseNet(nn.Module):
    def __init__(self):
        super(NoiseNet, self).__init__()
        self.model = models.vgg16(pretrained=True)
        self.model.classifier = nn.Sequential(nn.Linear(25088, 512),
                                  nn.ReLU(),
                                  nn.Dropout(0.25),
                                  nn.Linear(512,256),
                                  nn.ReLU(),
                                  nn.Dropout(0.5),
                                  nn.Linear(256, 1)
)

    def forward(self, x):
        return self.model(x)

model = NoiseNet()
model

NoiseNet(
  (model): VGG(
    (features): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (16):

In [20]:
#VGG16

from tensorboard.backend.event_processing.event_file_loader import EventFileLoader

# Initialize empty lists to store metrics
train_loss_list = []
test_loss_list = []
mae_list = []
rmse_list = []
r2_list = []

# Training and testing loop with cross-validation
epochs = 30
loss_fn = nn.MSELoss()


for fold, (train_idx, test_idx) in enumerate(kf.split(dataset)):
    print(f"Fold {fold + 1}")
    train_subset = torch.utils.data.Subset(dataset, train_idx)
    test_subset = torch.utils.data.Subset(dataset, test_idx)

    trainloader = DataLoader(train_subset, batch_size=128, shuffle=True)
    testloader = DataLoader(test_subset, batch_size=128, shuffle=False)

    model = NoiseNet().to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)

    custom_run_name = f"FT_Vgg_fold_{fold + 1}"
    writer = SummaryWriter(log_dir=f"runs/{custom_run_name}")

    for epoch in range(1, epochs + 1):
        train(model, trainloader, loss_fn, optimizer, epoch, writer)
        test(model, testloader, loss_fn, epoch, writer)

    writer.close()

    # Read scalar values from tensorboard event files

    event_file = glob.glob(f"runs/{custom_run_name}/events.out.tfevents.*")[0]
    event_loader = EventFileLoader(event_file)
    scalar_events = [event for event in event_loader.Load()]


    # Extract the required scalar values
    train_loss = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 1), None)
    test_loss = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 2), None)
    mae = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 3), None)
    rmse = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 4), None)
    r2 = next((event.summary.value[0].simple_value for event in scalar_events if event.step == 5), None)


    # Append the metrics to the respective lists
    train_loss_list.append(train_loss)
    test_loss_list.append(test_loss)
    mae_list.append(mae)
    rmse_list.append(rmse)
    r2_list.append(r2)



Fold 1
Train Epoch: 1 [0/8000 (0%)]	Loss: 0.026260
Train Epoch: 1	Average Loss: 4.293399
MAE: 0.400330, RMSE: 2.080313, R2 Score: -1358.178667
Test Epoch: 1	Average Loss: 0.005220
MAE: 0.059203, RMSE: 0.072227, R2 Score: -0.677815
Train Epoch: 2 [0/8000 (0%)]	Loss: 0.005771
Train Epoch: 2	Average Loss: 0.004013
MAE: 0.053063, RMSE: 0.063436, R2 Score: -0.263821
Test Epoch: 2	Average Loss: 0.003122
MAE: 0.048122, RMSE: 0.055798, R2 Score: -0.001355
Train Epoch: 3 [0/8000 (0%)]	Loss: 0.003219
Train Epoch: 3	Average Loss: 0.003820
MAE: 0.051953, RMSE: 0.061769, R2 Score: -0.198280
Test Epoch: 3	Average Loss: 0.003119
MAE: 0.048111, RMSE: 0.055769, R2 Score: -0.000306
Train Epoch: 4 [0/8000 (0%)]	Loss: 0.003203
Train Epoch: 4	Average Loss: 0.003732
MAE: 0.051666, RMSE: 0.061148, R2 Score: -0.174309
Test Epoch: 4	Average Loss: 0.003127
MAE: 0.048146, RMSE: 0.055846, R2 Score: -0.003056
Train Epoch: 5 [0/8000 (0%)]	Loss: 0.002993
Train Epoch: 5	Average Loss: 0.003730
MAE: 0.051647, RMSE: 0.0

In [ ]:
import numpy as np

# Remove None values from the lists
train_loss_list_cleaned = [x for x in train_loss_list if x is not None]
test_loss_list_cleaned = [x for x in test_loss_list if x is not None]
mae_list_cleaned = [x for x in mae_list if x is not None]
rmse_list_cleaned = [x for x in rmse_list if x is not None]
r2_list_cleaned = [x for x in r2_list if x is not None]

# Calculate mean scores using numpy
train_loss_mean = np.mean(train_loss_list_cleaned)
test_loss_mean = np.mean(test_loss_list_cleaned)
mae_mean = np.mean(mae_list_cleaned)
rmse_mean = np.mean(rmse_list_cleaned)
r2_mean = np.mean(r2_list_cleaned)

print("\nCross-Validation Results")
print(f"Train Loss - Mean: {train_loss_mean:.6f}")
print(f"Test Loss - Mean: {test_loss_mean:.6f}")
print(f"MAE - Mean: {mae_mean:.6f}")
print(f"RMSE - Mean: {rmse_mean:.6f}")
print(f"R2 Score - Mean: {r2_mean:.6f}")


In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [ ]:
  %reload_ext tensorboard